# Fuel Emissions Prediction

This task calls for the prediction of fuel emissions at 12.000 miles for different vehicles. We will train different
regression machine learning models and attempt to evaluate our solutions via relevant metrics

We begin with the necessary import, plus a random state for reproducible results

In [26]:
import pandas as pd
import numpy as np
from scipy.stats import skew
from sklearn import model_selection, metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from scipy import stats
from xgboost import XGBRegressor

RANDOM_STATE = 42

In [27]:
df = pd.read_csv("../input/fuel_emissions.csv")

/home/georg/miniconda3/envs/ML-Assignments/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Data analysis


In [28]:
df.head()

,file,year,manufacturer,model,description,euro_standard,tax_band,transmission,transmission_type,engine_capacity,...,thc_emissions,co_emissions,nox_emissions,thc_nox_emissions,particulates_emissions,fuel_cost_12000_miles,standard_12_months,standard_6_months,first_year_12_months,first_year_6_months
0,Part_A_Euro_IV_may2005.csv,2005,BMW,1 Series E87,116i,4,NaN,M5,Manual,1596.0,...,NaN,260.0,35.0,NaN,NaN,1158.0,NaN,NaN,NaN,NaN
1,Part_A_Euro_IV_may2005.csv,2005,BMW,1 Series E87,118d - from March 2005,4,NaN,M6,Manual,1995.0,...,NaN,290.0,166.0,NaN,17.0,422.0,NaN,NaN,NaN,NaN
2,Part_A_Euro_IV_may2005.csv,2005,BMW,1 Series E87,118d - up to February 2005,4,NaN,M6,Manual,1995.0,...,NaN,319.0,166.0,NaN,21.0,422.0,NaN,NaN,NaN,NaN
3,Part_A_Euro_IV_may2005.csv,2005,BMW,1 Series E87,118i,4,NaN,M5,Manual,1995.0,...,NaN,511.0,7.0,NaN,NaN,1128.0,NaN,NaN,NaN,NaN
4,Part_A_Euro_IV_may2005.csv,2005,BMW,1 Series E87,118i,4,NaN,A6,Automatic,1995.0,...,NaN,383.0,15.0,NaN,NaN,1206.0,NaN,NaN,NaN,NaN


In [29]:
df.info()
df.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33088 entries, 0 to 33087
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   file                    33088 non-null  object 
 1   year                    33088 non-null  int64  
 2   manufacturer            33088 non-null  object 
 3   model                   33088 non-null  object 
 4   description             33088 non-null  object 
 5   euro_standard           33088 non-null  int64  
 6   tax_band                7837 non-null   object 
 7   transmission            33078 non-null  object 
 8   transmission_type       32747 non-null  object 
 9   engine_capacity         33081 non-null  float64
 10  fuel_type               33088 non-null  object 
 11  urban_metric            33075 non-null  float64
 12  extra_urban_metric      33075 non-null  float64
 13  combined_metric         33081 non-null  float64
 14  urban_imperial          33075 non-null

file                         19
year                          9
manufacturer                 60
model                      2111
description               11811
euro_standard                 4
tax_band                     13
transmission                 74
transmission_type             2
engine_capacity             378
fuel_type                    13
urban_metric                245
extra_urban_metric          121
combined_metric             165
urban_imperial              323
extra_urban_imperial        296
combined_imperial           319
noise_level                  99
co2                         335
thc_emissions               304
co_emissions               1246
nox_emissions               821
thc_nox_emissions           185
particulates_emissions      279
fuel_cost_12000_miles      1841
standard_12_months           13
standard_6_months            10
first_year_12_months         10
first_year_6_months           3
dtype: int64

Our dataset consists of 33088 observations with 29 features. We can observe different types of features.
- Redundant features such as *file* or *description* can safely be dropped since they convey no relevant information
to the task
- There exist many categorical features, i.e. *manufacturer* or *transmission*

Certain features exhibit a wide range of values, like the categorical *model* that has over 2k discrete values and will
be challenging to be transformed


In [30]:
df.isnull().sum()

file                          0
year                          0
manufacturer                  0
model                         0
description                   0
euro_standard                 0
tax_band                  25251
transmission                 10
transmission_type           341
engine_capacity               7
fuel_type                     0
urban_metric                 13
extra_urban_metric           13
combined_metric               7
urban_imperial               13
extra_urban_imperial         13
combined_imperial             7
noise_level                   0
co2                           0
thc_emissions             16599
co_emissions                 24
nox_emissions                70
thc_nox_emissions         27658
particulates_emissions    19912
fuel_cost_12000_miles        10
standard_12_months        29571
standard_6_months         30162
first_year_12_months      29571
first_year_6_months       31669
dtype: int64

Our dataset has a large number of missing values on many features. We will attempt to better illustrate this as a
percentage of missing values, per column/feature


In [31]:
def get_missing(data):
    missing_percentage = (data.isnull().sum() / len(df)) * 100
    missing_df = pd.DataFrame({'Missing %': missing_percentage})
    missing_df =  missing_df.sort_values(by='Missing %', ascending=False)
    missing_df['Missing %'] = missing_df['Missing %'].round(2)

    return missing_df


In [32]:
missing = get_missing(df)
missing.head(29)


,Missing %
first_year_6_months,95.71
standard_6_months,91.16
first_year_12_months,89.37
standard_12_months,89.37
thc_nox_emissions,83.59
tax_band,76.31
particulates_emissions,60.18
thc_emissions,50.17
transmission_type,1.03
nox_emissions,0.21


As is easily observable a significant number of features misses over 50% of its values. There is no gain in
attempting to remedy so many missing values. So these features will be dropped, alongside the already mentioned
redundant-in-context features.


In [33]:
to_drop = [
    'file',
    'description',
    'first_year_6_months',
    'standard_6_months',
    'first_year_12_months',
    'standard_12_months',
    'thc_nox_emissions',
    'tax_band',
    'particulates_emissions',
    'thc_emissions'
]

df.drop(to_drop, axis=1, inplace=True)

In [34]:
missing = get_missing(df)
missing.head(20)

,Missing %
transmission_type,1.03
nox_emissions,0.21
co_emissions,0.07
extra_urban_metric,0.04
urban_metric,0.04
extra_urban_imperial,0.04
urban_imperial,0.04
fuel_cost_12000_miles,0.03
transmission,0.03
combined_metric,0.02


This leaves the dataset with a reduced number of features, but more manageable shortcomings. There still exist
missing values but at a rate that can be easily imputed/filled.

## Preprocessing
As a first step wil will imput our categorical features. Since they are not numerical, the most effective way to
impute them is by replicating the most frequent observation in each of these columns. Steps are taken so that only
the specific features are imputed in this manner.

In [35]:
transmission_type_df = df.filter(['transmission_type'], axis=1)
df.drop(['transmission_type'], axis=1, inplace=True)

imputer = SimpleImputer(strategy="most_frequent")
transmission_type = imputer.fit_transform(transmission_type_df)
df['transmission_type'] = transmission_type

In [36]:
transmission_df = df.filter(['transmission'], axis=1)
df.drop(['transmission'], axis=1, inplace=True)

imputer = SimpleImputer(strategy="most_frequent")
transmission = imputer.fit_transform(transmission_df)
df['transmission'] = transmission

In [37]:
missing = get_missing(df)
missing.head(29)

,Missing %
nox_emissions,0.21
co_emissions,0.07
urban_imperial,0.04
urban_metric,0.04
extra_urban_metric,0.04
extra_urban_imperial,0.04
fuel_cost_12000_miles,0.03
engine_capacity,0.02
combined_metric,0.02
combined_imperial,0.02


As the categorical features do not have any missing values, the next preprocessing step is to impute our numerical values.
This will be performed with the KNN imputer that utilizes the k-Nearest Neighbours to determine the missing values.
We will weight neighbours inversely by their distance to the observation.

To overcome KNNImputer's difficulties with categorical features, we will temporarily remove them and re-add them after
the imputation

In [38]:
categorical_labels = df.select_dtypes(include='object').columns
categorical_labels_df = df.filter(categorical_labels, axis=1)
df.drop(categorical_labels, axis=1, inplace=True)

In [39]:
imputer = KNNImputer(n_neighbors=5, weights='distance')
imputer.fit(df)

imputed_features = imputer.fit_transform(df.values)
imputed_df = pd.DataFrame(imputed_features, index=df.index, columns=df.columns)
imputed_df.head()

,year,euro_standard,engine_capacity,urban_metric,extra_urban_metric,combined_metric,urban_imperial,extra_urban_imperial,combined_imperial,noise_level,co2,co_emissions,nox_emissions,fuel_cost_12000_miles
0,2005.0,4.0,1596.0,10.5,5.9,7.5,26.9,47.9,37.7,73.0,181.0,260.0,35.0,1158.0
1,2005.0,4.0,1995.0,7.7,4.5,5.6,36.7,62.8,50.4,72.0,150.0,290.0,166.0,422.0
2,2005.0,4.0,1995.0,7.7,4.5,5.6,36.7,62.8,50.4,72.0,150.0,319.0,166.0,422.0
3,2005.0,4.0,1995.0,10.1,5.7,7.3,28.0,49.6,38.7,72.0,176.0,511.0,7.0,1128.0
4,2005.0,4.0,1995.0,10.7,6.1,7.8,26.4,46.3,36.2,70.0,188.0,383.0,15.0,1206.0


In [40]:
df = pd.concat([imputed_df, categorical_labels_df], axis=1)

missing = get_missing(df)
missing.head(18)

,Missing %
year,0.0
co2,0.0
transmission_type,0.0
fuel_type,0.0
model,0.0
manufacturer,0.0
fuel_cost_12000_miles,0.0
nox_emissions,0.0
co_emissions,0.0
noise_level,0.0


We can now observe that our dataset is now clean, withou any missing values. We can continue to the next step

## Outlier removal
In order to assist our regression models, we will attempt to remove observations that have extreme values in their
numerical features. We will do so by comparing to the 1st and 99th quantile and remove any observations that have
values outside of this range.

In [41]:
numerical_features = df.select_dtypes(exclude='object').columns

for feat in numerical_features:
    q_low = df[feat].quantile(0.01)
    q_hi  = df[feat].quantile(0.99)

    df = df[(df[feat] < q_hi) & (df[feat] > q_low)]

Next the categorical features have to be one-hot encoded and transformed to numerical. Additionally features need to be
scaled. A standard scaler is the best choice since we, *most importantly* have already removed any outliers and we
have no rate features


In [42]:
df = pd.get_dummies(df, columns=categorical_labels)
df.shape

(11364, 998)

In [43]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df.values)
scaled_df = pd.DataFrame(scaled_features, index=df.index, columns=df.columns)

## Model Training
After properly denoting our target and splitting our dataset to train and test slices we will train 3 different
regression models. A Linear Regression, a Random Forest and an XGBoost Regressor.

We will evaluate our models, using the spearman and pearson correlation and r2 score, where values as close to +1 as
possible point to a succesfull prediction. In addtion the mean and root square error will be calculated. Due to the range
of values of fuel cost, any error below 100 can be considered a success.


In [44]:
target = df['fuel_cost_12000_miles']
X = df.drop(['fuel_cost_12000_miles'], axis=1)
X.shape

(11364, 997)

In [45]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(X, target, test_size=0.25, random_state=RANDOM_STATE)

In [46]:
def evaluate_regression(actual, predicted):
    spearman_correlation = stats.spearmanr(actual, predicted)
    print(f'Spearman Correlation: {spearman_correlation.correlation:.2f}')

    pearson_correlation = stats.pearsonr(actual, predicted)
    print(f'Pearson Correlation: {pearson_correlation[0]:.2f}')

    r2_score = metrics.r2_score(actual, predicted)
    print(f'R2 Coefficient Score: {r2_score:.2f}')

    mean_squared_error = metrics.mean_squared_error(actual, predicted)
    print(f'Mean Squared Error: {mean_squared_error:.2f}')

    rmse = metrics.mean_squared_error(actual, predicted, squared=False)
    print(f'Root Mean Squared Error: {rmse:.2f}')

    # TODO needs upgraded scikit-learn
    # mape = metrics.mean_absolute_percentage_error(actual, predicted)
    # print(f'MAPE Score: {mape:.2f}')

The linear regression does not seem to be able to adequately describe the relation of the data and thus
predict our target variable correctly. On the contrary our ensemble methods, the Random Forest and XGBoost regressors
demonstrate a mean squared error around 80 and can be considered a success in predicting the emmissions.


In [47]:
lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)
y_pred = lin_reg.predict(x_test)
print("----Linear Regression----")
evaluate_regression(y_test, y_pred)

----Linear Regression----
Spearman Correlation: 0.90
Pearson Correlation: -0.02
R2 Coefficient Score: -1738734349009.30
Mean Squared Error: 144757970247679040.00
Root Mean Squared Error: 380470721.93


In [48]:
rf_reg = RandomForestRegressor()
rf_reg.fit(x_train, y_train)
y_pred = rf_reg.predict(x_test)
print("----Random Forest Regression----")
evaluate_regression(y_test, y_pred)

----Random Forest Regression----
Spearman Correlation: 1.00
Pearson Correlation: 1.00
R2 Coefficient Score: 1.00
Mean Squared Error: 80.66
Root Mean Squared Error: 8.98


In [49]:
xgb_reg = XGBRegressor(gamma=0, objective='reg:linear', nthread=-1)
xgb_reg.fit(x_train, y_train)
y_pred = xgb_reg.predict(x_test)
print("----XGBoost Regression----")
evaluate_regression(y_test, y_pred)


[00:52:43] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
----XGBoost Regression----
Spearman Correlation: 1.00
Pearson Correlation: 1.00
R2 Coefficient Score: 1.00
Mean Squared Error: 81.41
Root Mean Squared Error: 9.02
